# Modelos de Predicción
##### Futuras mejoras
* Estudiar bien los métodos de clasificación
* Cambiar el KNearestNeigh.. pues no está bien implementado: Falta normalizar, definir métrica, etc.
* Ordenar por tiempo y separar
* hacer crossvalidation
* 

## Importación de librerías

In [1]:
from os import path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

## Constantes

Rutas usuales que se ocuparán en el notebook

In [2]:
# Salvar gráficos
save_graf = False

# Variable que comenta el estado actual de las funciones
MOSTRAR_INFO = False

# RUTAS
MAIN_PATH = path.join("..")

IMG_PATH = path.join(MAIN_PATH, "imagenes")
DATA_PATH = path.join(MAIN_PATH, "data")
INF_PATH = path.join(MAIN_PATH, "informe")

TAB_PATH = path.join(INF_PATH, "tablas")

WF_FOLDER_PATH = path.join(DATA_PATH,
                           "wildfires_us")
WF_DATA_PATH = path.join(WF_FOLDER_PATH,
                         "WILDFIRES_USA.csv")
WF_DATA_COLUMNS_PATH = path.join(WF_FOLDER_PATH,
                                 "WILDFIRES_USA_COLUMNS.csv")

# Tamaño de la imagen
my_figsize = (10, 5)

## Funciones auxiliares

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [3]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [4]:
# Todas las columnas
columnas = str(pd.read_csv(WF_DATA_COLUMNS_PATH).columns[0]).split(",")

# Columnas que se ocuparán en el análisis
columnas_ocupadas = columnas.copy()

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [5]:
df = pd.read_csv(WF_DATA_PATH)

# Convertimos los datos que sean fechas en ese tipo de dato
df["DISC_DATE_TIME"] = pd.to_datetime(df["DISC_DATE_TIME"])
df["CONT_DATE_TIME"] = pd.to_datetime(df["CONT_DATE_TIME"])

print_cantidad(df)

df.head()

El dataset tiene una cantidad de 890821 datos y 16 variables.


,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,1992,Miscellaneous,0.36,B,32.4410,-82.7018,GA,1992-01-01 00:01:00,January,Wednesday,0,1992-01-01 01:14:00,January,Wednesday,1,1.216667
1,1992,Miscellaneous,1.02,B,32.2009,-82.2967,GA,1992-01-01 00:05:00,January,Wednesday,0,1992-01-01 00:40:00,January,Wednesday,0,0.583333
2,1992,Miscellaneous,0.58,B,32.1325,-82.7610,GA,1992-01-01 00:10:00,January,Wednesday,0,1992-01-01 01:15:00,January,Wednesday,1,1.083333
3,1992,Children,10.14,C,32.0300,-83.9350,GA,1992-01-01 00:30:00,January,Wednesday,0,1992-01-01 02:20:00,January,Wednesday,2,1.833333
4,1992,Lightning,0.10,A,38.2050,-120.3350,CA,1992-01-01 01:30:00,January,Wednesday,1,1992-01-01 02:10:00,January,Wednesday,2,0.666667


## Preparación de la data

### Creación de las tablas para los distintos escenarios

In [6]:
# Copia del dataset
df_1 = df.copy()
df_1.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,1992,Miscellaneous,0.36,B,32.4410,-82.7018,GA,1992-01-01 00:01:00,January,Wednesday,0,1992-01-01 01:14:00,January,Wednesday,1,1.216667
1,1992,Miscellaneous,1.02,B,32.2009,-82.2967,GA,1992-01-01 00:05:00,January,Wednesday,0,1992-01-01 00:40:00,January,Wednesday,0,0.583333
2,1992,Miscellaneous,0.58,B,32.1325,-82.7610,GA,1992-01-01 00:10:00,January,Wednesday,0,1992-01-01 01:15:00,January,Wednesday,1,1.083333
3,1992,Children,10.14,C,32.0300,-83.9350,GA,1992-01-01 00:30:00,January,Wednesday,0,1992-01-01 02:20:00,January,Wednesday,2,1.833333
4,1992,Lightning,0.10,A,38.2050,-120.3350,CA,1992-01-01 01:30:00,January,Wednesday,1,1992-01-01 02:10:00,January,Wednesday,2,0.666667


In [7]:
df_2 = df.copy()
df_2 = df_2[ df_2["FIRE_YEAR"] >= 2011 ]
df_2.reset_index(drop=True, inplace=True)
df_2.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2011,Miscellaneous,1.00,B,38.198567,-93.002533,MO,2011-01-01 00:01:00,January,Saturday,0,2011-01-31 23:59:00,January,Monday,23,743.966667
1,2011,Fireworks,0.30,B,28.105500,-82.517700,FL,2011-01-01 00:01:00,January,Saturday,0,2011-01-01 03:35:00,January,Saturday,3,3.566667
2,2011,Miscellaneous,0.10,A,32.927200,-115.127000,CA,2011-01-01 00:09:00,January,Saturday,0,2011-01-01 00:25:00,January,Saturday,0,0.266667
3,2011,Children,0.18,A,33.187149,-84.918798,GA,2011-01-01 00:10:00,January,Saturday,0,2011-01-01 01:36:00,January,Saturday,1,1.433333
4,2011,Arson,5.00,B,33.464155,-85.996639,AL,2011-01-01 00:15:00,January,Saturday,0,2011-01-01 02:50:00,January,Saturday,2,2.583333


In [8]:
def new_category(cause):
    """Re-categoriza las causas que se tenían originalmente.
    """
    d_causes = {
        "Other": ['Miscellaneous',
                  'Missing/Undefined'],
        "Human": ['Children',
                  'Smoking',
                  'Equipment Use',
                  'Debris Burning',
                  'Campfire',
                  'Campfire',
                  'Railroad',
                  'Powerline',
                  'Fireworks',
                  'Structure'],
        "Natural": ['Lightning'],
        "Malicious": ['Arson']
    }

    for new_cause in d_causes:
        if cause in d_causes[new_cause]:
            return new_cause
    return None

In [9]:
df_3 = df_2.copy()
df_3["STAT_CAUSE_DESCR"] = df_3.STAT_CAUSE_DESCR.map(new_category)
df_3.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2011,Other,1.00,B,38.198567,-93.002533,MO,2011-01-01 00:01:00,January,Saturday,0,2011-01-31 23:59:00,January,Monday,23,743.966667
1,2011,Human,0.30,B,28.105500,-82.517700,FL,2011-01-01 00:01:00,January,Saturday,0,2011-01-01 03:35:00,January,Saturday,3,3.566667
2,2011,Other,0.10,A,32.927200,-115.127000,CA,2011-01-01 00:09:00,January,Saturday,0,2011-01-01 00:25:00,January,Saturday,0,0.266667
3,2011,Human,0.18,A,33.187149,-84.918798,GA,2011-01-01 00:10:00,January,Saturday,0,2011-01-01 01:36:00,January,Saturday,1,1.433333
4,2011,Malicious,5.00,B,33.464155,-85.996639,AL,2011-01-01 00:15:00,January,Saturday,0,2011-01-01 02:50:00,January,Saturday,2,2.583333


### Convertir variables categóricas a númericas

In [10]:
causas = df["STAT_CAUSE_DESCR"].unique().tolist()
causas

['Miscellaneous',
 'Children',
 'Lightning',
 'Smoking',
 'Arson',
 'Equipment Use',
 'Debris Burning',
 'Campfire',
 'Railroad',
 'Missing/Undefined',
 'Powerline',
 'Fireworks',
 'Structure']

In [11]:
# Columnas que no se ocuparán
drop_list = ["DISC_DATE_TIME",
             "CONT_DATE_TIME",
             "FIRE_SIZE_CLASS", 
             "FIRE_YEAR", 
             "CONT_MONTH",
             "CONT_DOW"
            ]

In [12]:
# Eliminamos las columnas que no se ocuparán
for to_drop in drop_list:
    df_1 = df_1.drop(to_drop, axis=1)
    df_2 = df_2.drop(to_drop, axis=1)
    df_3 = df_3.drop(to_drop, axis=1)
df_1.head()

,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_TIME,DT_FIRE
0,Miscellaneous,0.36,32.4410,-82.7018,GA,January,Wednesday,0,1,1.216667
1,Miscellaneous,1.02,32.2009,-82.2967,GA,January,Wednesday,0,0,0.583333
2,Miscellaneous,0.58,32.1325,-82.7610,GA,January,Wednesday,0,1,1.083333
3,Children,10.14,32.0300,-83.9350,GA,January,Wednesday,0,2,1.833333
4,Lightning,0.10,38.2050,-120.3350,CA,January,Wednesday,1,2,0.666667


In [13]:
cols = [
    "STATE", 
    "DISC_MONTH",
    "DISC_DOW",
    "STAT_CAUSE_DESCR",
]

In [14]:
le_list = [preprocessing.LabelEncoder() for _ in range(3)]
df_list = [df_1, df_2, df_3]

In [15]:
for col in cols:
    for i in range(3):
        df_ = df_list[i]
        le_ = le_list[i]
        df_[col] = le_.fit_transform(df_[col])
#     df_[col] = le.fit_transform(df_[col])
    
df_.head()

,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_TIME,DT_FIRE
0,3,1.00,38.198567,-93.002533,24,4,2,0,23,743.966667
1,0,0.30,28.105500,-82.517700,9,4,2,0,3,3.566667
2,3,0.10,32.927200,-115.127000,4,4,2,0,0,0.266667
3,0,0.18,33.187149,-84.918798,10,4,2,0,1,1.433333
4,1,5.00,33.464155,-85.996639,1,4,2,0,2,2.583333


## Creación de funciones auxiliares para los modelos

In [16]:
def save_classification_report(y_true, y_pred,
                               target_names=None,
                               buf=None, label=None, caption=None,
                               support=True, verbose=False, escenario_name=None,):
    """Salva el reporte en formato LaTeX.
    """
    # Importaciones básicas
    from sklearn.metrics import classification_report
    import pandas as pd
    from numpy import nan as NA
    from os import path
    
    label = label if label is None else f"tab:{label}"
    
    if verbose: print("Creando Reporte...")
    # Guardamos el reporte de clasificación en formato de diccionario
    d = classification_report(y_true, y_pred,
                              target_names=target_names,
                              output_dict=True)
    
    # Creamos una función que pasa el reporte a DataFrame
    def dicc2df(d):
        d = d.copy()
        d['accuracy'] = {'precision': NA,
                         'recall': NA,
                         'f1-score': d['accuracy'],
                         'support': d['weighted avg']['support']}
        df = pd.DataFrame(d)
        df = df.T
        df["support"] = df["support"].astype('int32')
        return df
    
    df = dicc2df(d)
    
    # Si no se quiere que se entregue el soporte
    if not support: df.pop("support")
    if verbose: print("Reporte Creado!")
    
    if verbose: print("Creando LaTeX...")
    # Guardamos el dataframe
    df_latex = df.to_latex(buf=buf,
                           na_rep="",
                           float_format="%.3f",
                           caption=caption,
                           label=label,)
    if verbose: print("LaTeX creado!")
    
    if df_latex is not None and verbose:
        print("El reporte en latex es el siguiente:\n")
        print(df_latex)
    
    return df

Se crearan distintas funciones para predecir

### Modelo Naïve Bayes

In [17]:
def modelo_naive_bayes(X_tr, y_tr, X_te, y_te,
                       verbose=MOSTRAR_INFO, target_names=None,
                       escenario_name=None,
                      ):
    """Genera el modelo de Naïve Bayes e imprime el resumen de clasificación
    """
    from sklearn.naive_bayes import GaussianNB
    
    if verbose: print("Creando modelo...")
    model = GaussianNB()
    
    if verbose: print("Ajustando modelo...")
    model.fit(X_tr, y_tr)
    
    if verbose: print("Prediciendo...")
    y_pred = model.predict(X_te)
    
    if verbose: print(classification_report(y_te, y_pred, target_names=target_names))
    score = model.score(X_te, y_te)
    
    save_classification_report(y_te, y_pred,
                               support=False, verbose=verbose,
                               buf=path.join(TAB_PATH, 
                                             f"NB_{escenario_name}.tex"),
                               caption=("Reporte de clasificación"
                                      f" para el {escenario_name}"
                                      " utilizando Naïve Bayes"),
                               label=f"NB_{escenario_name}",
                               target_names=target_names
                              )
    
    return score

### Modelo K-NN

Se utiliza una función auxiliar para encontrar el $k$ óptimo. Si el $k$ óptimo ya se encontró, devuelve ese valor.

In [18]:
k_predicho, k_optimo = True, 40

def buscar_k(X_tr, y_tr, X_te, y_te,
             k_range=None, k_min=1, k_max=20, gap=1, 
             verbose=MOSTRAR_INFO):
    """Función utiliazada para determinar el k óptimo.
    """
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    global k_predicho
    global k_optimo
    
    # Si ya había un k óptimo, retorna el valor de k
    if k_predicho:
        if verbose: print(f"El k óptimo es:{k_optimo}")
        return k_optimo
    
    if verbose: print("Normalizando...")
    # Escalamos
    scaler = MinMaxScaler()
    X_tr_scaled = (scaler.fit_transform(X_tr))
    X_te_scaled = (scaler.transform(X_te))
    
    if verbose: print("Buscando k óptimo...")
    # Vemos los distintos valores de k
    k_range = range(k_min, k_max+1, gap) if not k_range else k_range
    k_range = list(k_range)
    scores = dict()
    for k in k_range:
        if verbose: print(f"Entrenando con k={k}")
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_tr_scaled, y_tr)
        score = knn.score(X_te_scaled, y_te)
        scores[k] = score
#         if i > 0 and scores[k_range[i-1]] < scores[k_range[i]]:
#             break
        if verbose: print(f"Score para k={k}:{score}\n" + "="*40)
    
    k_optimo = max(k_range, key=lambda x: scores[x])
    k_predicho = True
    
    if verbose: print(f"El k óptimo es:{k_optimo}")
    
    return k_optimo

In [19]:
def modelo_knn(X_tr, y_tr, X_te, y_te, 
               verbose=MOSTRAR_INFO, target_names=None,
               normalizar=True, buscar_k_optimo=not k_predicho, escenario_name=None,):
    """Genera el modelo de Naïve Bayes e imprime el resumen de clasificación.
    """
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    global k_predicho
    global k_optimo
    
    if verbose and buscar_k_optimo: print("Buscando k óptimo...")
    if buscar_k_optimo:
        k_predicho = False
        k_optimo = None
    k = buscar_k(X_tr, y_tr, X_te, y_te, verbose=False)
    
    if verbose and normalizar: print("Normalizando datos...")
    scaler = MinMaxScaler()
    X_tr = (scaler.fit_transform(X_tr)) if normalizar else X_tr
    X_te = (scaler.transform(X_te)) if normalizar else X_te
    
    if verbose: print("Creando modelo...")
    model = KNeighborsClassifier(n_neighbors=k)
    
    if verbose: print("Ajustando modelo...")
    model.fit(X_tr, y_tr)
    
    if verbose: print("Prediciendo...")
    y_pred = model.predict(X_te)
    
    if verbose: print(classification_report(y_te, y_pred, target_names=target_names))
    score = model.score(X_te, y_te)
    
    save_classification_report(y_te, y_pred,
                               support=False, verbose=verbose,
                               buf=path.join(TAB_PATH, 
                                             f"KNN_{escenario_name}.tex"),
                               caption=("Reporte de clasificación"
                                      f" para el {escenario_name}"
                                      " utilizando KNN"),
                               label=f"KNN_{escenario_name}",
                               target_names=target_names
                              )
    
    return score

### Modelo de Discriminante Lineal

In [20]:
def modelo_discr_lineal(X_tr, y_tr, X_te, y_te,
                        verbose=MOSTRAR_INFO, target_names=None, escenario_name=None,):
    """Genera el modelo de Discriminante Lineal e imprime el resumen de clasificación
    """
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    
    if verbose: print("Creando modelo...")
    model = LinearDiscriminantAnalysis()
    
    if verbose: print("Ajustando modelo...")
    model.fit(X_tr, y_tr)
    
    if verbose: print("Prediciendo...")
    y_pred = model.predict(X_te)
    
    if verbose: print(classification_report(y_te, y_pred, target_names=target_names))
    score = model.score(X_te, y_te)
    
    save_classification_report(y_te, y_pred,
                               support=False, verbose=verbose,
                               buf=path.join(TAB_PATH, 
                                             f"DL_{escenario_name}.tex"),
                               caption=("Reporte de clasificación"
                                      f" para el {escenario_name}"
                                      " utilizando Discriminante Lineal"),
                               label=f"DL_{escenario_name}",
                               target_names=target_names
                              )
    
    return score

### Modelo de Discriminante Cuadrático

In [21]:
def modelo_discr_cuadr(X_tr, y_tr, X_te, y_te,
                       verbose=MOSTRAR_INFO, target_names=None, escenario_name=None,):
    """Genera el modelo de Discriminante Cuadrático e imprime el resumen de clasificación
    """
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    
    if verbose: print("Creando modelo...")
    model = QuadraticDiscriminantAnalysis()
    
    if verbose: print("Ajustando modelo...")
    model.fit(X_tr, y_tr)
    
    if verbose: print("Prediciendo...")
    y_pred = model.predict(X_te)
    
    if verbose: print(classification_report(y_te, y_pred, target_names=target_names))
    score = model.score(X_te, y_te)
    
    save_classification_report(y_te, y_pred,
                               support=False, verbose=verbose,
                               buf=path.join(TAB_PATH, 
                                             f"DC_{escenario_name}.tex"),
                               caption=("Reporte de clasificación"
                                      f" para el {escenario_name}"
                                      " utilizando Discriminante Cuadrático"),
                               label=f"DC_{escenario_name}",
                               target_names=target_names
                              )
    
    return score

### Modelo de Random Forest

In [22]:
def modelo_random_forest(X_tr, y_tr, X_te, y_te,
                         verbose=MOSTRAR_INFO, target_names=None, escenario_name=None,):
    """Genera el modelo de Random Forest e imprime el resumen de clasificación
    """
    from sklearn.ensemble import RandomForestClassifier
    
    if verbose: print("Creando modelo...")
    model = RandomForestClassifier(n_jobs=4,
                                   random_state=3435)
    
    if verbose: print("Ajustando modelo...")
    model.fit(X_tr, y_tr)
    
    if verbose: print("Prediciendo...")
    y_pred = model.predict(X_te)
    
    if verbose: print(classification_report(y_te, y_pred, target_names=target_names))
    score = model.score(X_te, y_te)
    
    save_classification_report(y_te, y_pred,
                               support=False, verbose=verbose,
                               buf=path.join(TAB_PATH, 
                                             f"RF_{escenario_name}.tex"),
                               caption=("Reporte de clasificación"
                                      f" para el {escenario_name}"
                                      " utilizando Random Forest"),
                               label=f"RF_{escenario_name}",
                               target_names=target_names
                              )
    
    return score

## Resumen de modelos

In [23]:
MODELOS = {
    "Discriminante_Cuadratico": modelo_discr_cuadr,
    "Discriminante_Lineal": modelo_discr_lineal,
    "KNN": modelo_knn,
    "Naive_Bayes": modelo_naive_bayes,
    "Random_Forest": modelo_random_forest,
}

def resumen_modelos(X_tr, y_tr, X_te, y_te,
                    verbose=MOSTRAR_INFO, target_names=None,
                    escenario_name=None, save_table=False
                   ):
    """Resume los modelos
    """
    
    data_ = (X_tr, y_tr, X_te, y_te)
    
    L_nombres = []
    L_scores = []
    for name in MODELOS:
        if verbose: print(f"Se está prediciendo el modelo {name}")
        L_nombres.append(name)
        score = round(MODELOS[name](*data_, 
                                    verbose=verbose, 
                                    target_names=target_names,
                                    escenario_name=escenario_name), 4)
        L_scores.append(score)
        if verbose: print("="*40)
        
        
    d_scores = {
        "Nombres": L_nombres,
        "Scores": L_scores
    }
    
    d_scores = pd.DataFrame(d_scores)
    d_scores.set_index("Nombres", inplace=True)
    d_scores.sort_values(by="Scores", inplace=True)
    
    return d_scores

### Primer Escenario

#### Preparación de la data de entrenamiento con la de prueba

In [24]:
X_1 = df_1.drop("STAT_CAUSE_DESCR", axis=1) # .values
y_1 = df_1.STAT_CAUSE_DESCR # .values

In [25]:
target_names_1 = list(le_list[0].classes_)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_1,
                                                    y_1,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    shuffle=False
                                                   )
data_1 = (X_train, y_train, X_test, y_test)

#### Prueba del primer escenario

In [27]:
escenario_1 = resumen_modelos(*data_1,
                              target_names=target_names_1,
                              escenario_name="Escenario 1")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [28]:
escenario_1

,Scores
Nombres,
Naive_Bayes,0.2215
Discriminante_Cuadratico,0.2451
Discriminante_Lineal,0.3465
KNN,0.3978
Random_Forest,0.4562


## Segundo Escenario

In [29]:
X_2 = df_2.drop("STAT_CAUSE_DESCR", axis=1) # .values
y_2 = df_2.STAT_CAUSE_DESCR # .values

In [30]:
target_names_2 = list(le_list[1].classes_)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_2,
                                                    y_2,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    shuffle=False
                                                   )
data_2 = (X_train, y_train, X_test, y_test)

In [32]:
escenario_2 = resumen_modelos(*data_2,
                              target_names=target_names_2,
                              escenario_name="Escenario 2")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [33]:
escenario_2

,Scores
Nombres,
Naive_Bayes,0.1400
Discriminante_Cuadratico,0.1512
Discriminante_Lineal,0.3233
KNN,0.4266
Random_Forest,0.5277


## Tercer Escenario

In [34]:
X_3 = df_3.drop("STAT_CAUSE_DESCR", axis=1) # .values
y_3 = df_3.STAT_CAUSE_DESCR # .values

In [35]:
target_names_3 = list(le_list[2].classes_)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_3,
                                                    y_3,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    shuffle=False
                                                   )
data_3 = (X_train, y_train, X_test, y_test)

In [ ]:
escenario_3 = resumen_modelos(*data_3,
                              target_names=target_names_3,
                              escenario_name="Escenario 3")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
escenario_3

In [ ]:
escenario_1.rename(columns={"Scores":"Escenario 1"}, inplace=True)
escenario_2.rename(columns={"Scores":"Escenario 2"}, inplace=True)
escenario_3.rename(columns={"Scores":"Escenario 3"}, inplace=True)

resumen = pd.concat([escenario_1, escenario_2, escenario_3],
                    axis=1,
                    sort=False)
resumen.index.rename("Modelos", inplace=True)

resumen

In [ ]:
resumen["Nombres"] = resumen.index
resumen = resumen[["Nombres", "Experimento 1", "Experimento 2", "Experimento 3"]]
resumen

In [ ]:
resumen_save_name = path.join(TAB_PATH, "resumen_experimentos.tex")
res = resumen.to_latex(index=True,
                       buf=resumen_save_name,
                       caption="Tabla resumen de los modelos",
                       label="tab:resumen-exp",
                       column_format="rccc"
                       )
print(res)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# # Entrenamos
# rfc = RandomForestClassifier(n_estimators=100, class_weight= "balanced", verbose=1, max_depth=5) # class_weight="balanced_subsample"
# rfc = rfc.fit(X_train, y_train)

# # Predecimos
# y_pred_rfc = rfc.predict(X_test)

In [ ]:
# import multiprocessing

# multiprocessing.cpu_count()

In [ ]:
# data = (X_train, y_train, X_test, y_test)
# modelo_discr_cuadr(*data)

### Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# # Entrenamos
# rfc = RandomForestClassifier(n_estimators=100, class_weight= "balanced", verbose=1, max_depth=5) # class_weight="balanced_subsample"
# rfc = rfc.fit(X_train, y_train)

# # Predecimos
# y_pred_rfc = rfc.predict(X_test)

In [ ]:
# # Mostramos
# print(classification_report(y_test, y_pred_rfc))

In [ ]:
# # Entrenamos
# rfc = RandomForestClassifier(n_estimators=50) # class_weight="balanced_subsample"
# rfc = rfc.fit(X_train, y_train)

# # Predecimos
# y_pred_rfc = rfc.predict(X_test)

In [ ]:
# # Mostramos
# print(classification_report(y_test, y_pred_rfc))